# **Convertidor de bases numéricas**

Daniel López  
Computación 8093

---
---

### **Base-n a Base-10.**

La primera mitad del problema consiste en tomar un entero de la forma $(r_k \ldots r_1r_0)_n$ y convertirlo a una expresión decimal. Para ello primero expandimos el polinomio:

$$(r_k \ldots r_1r_0)_n = r_k \cdot n^k \ldots r_1 \cdot n + r_0 \cdot n^0 .$$

Aquí es donde nos encontramos con el primer problema, pues en bases mayores a $10$ se necesitan símbolos adicionales a los números indoarábigos para denotar los coeficientes. En matemáticas suele usarse el número decimal entre paréntesis; algo como $((11)(12)8)_{13}$, pero es complicado traducir esta notación a código.

Otra solución común es asignarle un valor numérico a cada letra del alfabeto, por ejemplo $a=10$, $b=11$, etc. En mi caso simplemente opté por trabajar con bases menores a 10.

---

#### *Información necesaria del polinomio*

In [ ]:
base = int(input('Base: '))

if base > 10:
    print("Base no soportada.")
    exit()

bin = list(input('Número: '))
order = int(len(bin)) - 1
filter = all(int(x) < base for x in bin)

Lo primero que se pide es la base en la que está escrito el número; si es mayor a 10 el proceso se detiene.
Posteriormente se pide el número a convertir, que se almecena dígito a dígito en una lista llamada `bin`. 

Además, se almacena la longitud del número (su cantidad de dígitos - 1), y un valor *booleano* en la variable `filter`, el cual verifica que todos los coeficientes sean estrictamente menores a la base, pues de lo contrario, el número no tendría sentido en la base especificada anteriormente.

In [ ]:
num = []
ind = []
pow = []
times = []

El método que implementé se basa en las listas de Python, aquí simplemente se les da nombres convenientes.

---

#### *El algoritmo*

In [ ]:
if filter == True:
    for i, x in enumerate(bin):
        if x != '0':                                # Almacena la posición de los dígitos que no son 0.
            ind.append(i)

Lo primero que se hace es almacenar el **índice** de los dígitos (su posición dentro de `bin`) que no son 0 en la lista `ind`. Excluir los ceros no es necesario (de hecho es más sencillo no hacerlo), pero considero que vale la pena para agilizar los cálculos, en especial si se tienen bases pequeñas y/o números grandes. 

La posición del dígito es importante, pues nos dice  qué potencia de la base debe multiplicar. Nótese también que si el número tiene $n$ dígitos, los índices solo llegan hasta $k = n-1$, porque empiezan desde el 0.
Además, se almacenan en orden inverso; es decir, el coeficiente $r_k$ tiene la posición 0 dentro de la lista, pues es el que aparece primero. De la misma forma, el $r_{k-1}$ está en la posición 1 y, en general, el $r_i$ está en la posición $n-1-i = k-i$.

In [ ]:
for x in ind:
    pow.append(base**(order - x))               # Almacena las potencias cuyos coeficientes no son 0.

Una vez excluidos los ceros, necesitamos saber qué potencias de la base multiplicará cada dígito. Para ello simplemente traducimos a código lo que se explicó arriba; la k-ésima potencia de la base está en la posición $n-1-i$. El valor $n-1$ se almacenó previamente en la variable `order`. 

Además, usamos solo los índices de los númeron que no son cero. Por ejemplo, en el caso del número binario $1001101$ la lista almacenaría los valores $[2^6, 2^3, 2^2, 2^0]$.

In [ ]:
for x in bin:
    if x != '0':                                # Almacena los dígitos que no son 0.
        num.append(int(x))

Ahora, se almacenan los coeficientes no nulos en la lista `num`, esta vez como números operables, pues en `bin` eran strings.

In [ ]:
for j in range(len(num)):                       # Multiplica todas las potencias con sus
    times.append(num[j] * pow[j])               # respectivos coeficientes.
    
print('\n' + str(sum(times)) + '\n')

Por la forma en la que se escribió el algoritmo, las listas `num` y `pow` ahora contienen respectivamente los coeficientes y las potencias de la base, **en el mismo orden**. Además, dichas listas tienen la misma longitud, pues en ambas se excluyeron los ceros. 

Entonces basta con multiplicar cada potencia con su coeficiente (cuyos valores se almacenan en `times`) y luego sumar todos los resultados. El valor final es el número en notación decimal.

---
---

### **Base-10 a Base-n**

La segunda parte del problema es la operación inversa, tomar un número decimal y convertirlo a cualquier otra base de nuestra elección. Para esto, podemos aplicar iteradamente el algoritmo de la división sobre los cocientes del número, dividiendo entre la base y escribiendo el resultado en la forma $n = qb + r$, donde $b$ es la *base*, $q$ el *cociente* y $r$ el *residuo*.

Por ejemplo, si tenemos $273$ y queremos convertirlo a Base-5, escribimos

\begin{align*}
	273 &= 54\cdot 5 + 3 \\
	&= (10\cdot 5 + 4)\cdot 5 + 3 \cdot 5^0 \\
	&= 10\cdot 5^2 + 4\cdot 5 + 3 \cdot 5^0 \\
	&= 2\cdot 5^3 + 0\cdot 5^2 + 4\cdot 5 + 3 \cdot 5^0. 
\end{align*}

Así, $273 = (2043)_5$. 

Esto puede traducirse fácilmente a código usando un **while**, pues el proceso anterior termina cuando el último cociente $r_k$ sea estrictamente menor a la base.

---

#### *Trabajo preliminar.*

In [ ]:
def reverse(list):                                   # Invierte el orden de elementos en una lista.
    return [x for x in reversed(list)]

def cat(list):                                       # Concatena los elementos de una lista en un string.
    blank = ''
    return blank.join(list)

Lo primero que se hace es definir las funciones que se usarán en el algoritmo. En este caso solo se necesitan algunos métodos básicos para manipular listas.

In [ ]:
num = int(input('Número: '))
base = int(input('Base: '))

if base > 10:
    print("Base no soportada.")
    exit()
    
mod = []

Posteriormente se obtiene la información necesaria del usuario, o sea, el número a convertir y la base en la que se quiere el resultado. También se declara la lista que se usará en el algoritmo.

De nuevo, si se ingresa una base mayor a 10, el proceso se aborta.

---

#### *El algoritmo.*

Este es mucho más sencillo que el anterior, ya que puede resumirse en una sola instrucción:

In [ ]:
while base <= num:
    mod.append(str(num % base))
    num = num // base                            # Equivale a floor(num / base).
mod.append(str(num))

Es básicamente la traducción a código de lo que se dijo al inicio; la información que nos interesa son los residuos, pues estos son precisamente la representación del número en la base deseada. En python pueden obtenerse directamente con el módulo `%`.

Después de almacenar el residuo, actualizamos el número al cociente que resulta de dividir entre la base, pues el algoritmo de la división se aplica iteradamente sobre ellos. Este paso en particular puede generar problemas con números grandes si se hace la división normal con flotantes `/`, pues Python redondea el resultado (me pasó).

Como lo que queremos en este caso es el cociente entero y no la división exacta, podemos usar simplemente `//`, que indica división de enteros. Este método es equivalente a $\left\lfloor \frac{a}{b} \right\rfloor$.

Una vez que el **while** termina, el último cociente queda almacenado en `num`, así que también debemos agregarlo a la lista `mod`.

In [ ]:
convert = reverse(mod)
print('\n' + cat(convert) + '\n')

Por último, revertimos el orden de `mod` (pues los residuos se almacenaron al revés) e imprimimos el número resultante a la pantalla, que es la representación en la base deseada.